# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities6.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,oranjemund,-28.5500,16.4333,17.23,74,100,4.73,NaN,1697029565
1,1,sao francisco,-15.9486,-44.8644,32.47,35,4,4.42,BR,1697029567
2,2,longyearbyen,78.2186,15.6401,-4.09,68,20,5.14,SJ,1697029313
3,3,port alfred,-33.5906,26.8910,21.21,83,85,0.89,ZA,1697029361
4,4,lazaro cardenas,17.9583,-102.2000,25.60,90,99,2.56,MX,1697029569


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
city_plot = city_data_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles = "OSM", size = "Humidity", color = "City", scale = 1, frame_width = 900, frame_height = 600
)
city_plot
# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
my_ideal_weather = pd.DataFrame(city_data_df.loc[(city_data_df['Max Temp'] <= 75)])

# Drop any rows with null values


In [10]:

# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE

my_ideal_weather = pd.DataFrame(city_data_df.loc[(city_data_df['Max Temp'] >= 25)])
my_ideal_weather
my_ideal_weather.dropna(subset=['Country'], inplace = True)



In [11]:
my_ideal_weather.isna().sum()


City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       0
Date          0
dtype: int64

In [46]:
%%capture --no-display

# Configure the map plot
ideal_weather = my_ideal_weather.hvplot.points(
    "Lng", "Lat", geo=True, tiles = "OSM", size = "Max Temp", color = "City", scale = 1, frame_width = 900, frame_height = 600
)
ideal_weather

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp)

### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = my_ideal_weather[['City','Country', 'Lng', 'Lat', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df['Hotel Name'] = ""
# Display sample data
# YOUR CODE HERE
hotel_df.head(25)

,City,Country,Lng,Lat,Humidity,Hotel Name
1,sao francisco,BR,-44.8644,-15.9486,35,
4,lazaro cardenas,MX,-102.2000,17.9583,90,
7,enewetak,MH,162.3373,11.3474,77,
8,bac lieu,VN,105.7244,9.2850,82,
10,lata,PT,-8.3327,40.1629,37,
15,nouadhibou,MR,-17.0347,20.9310,20,
22,tanout,NE,8.8879,14.9709,13,
24,arraial do cabo,BR,-42.0278,-22.9661,65,
48,eraan,PH,117.7016,9.0629,82,
51,misratah,LY,15.0925,32.3754,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000

params = {"categories": "accommodation.hotel", "apiKey": geoapify_key}

    # YOUR CODE HERE

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = hotel_df.loc[index, "Lat"]
    lon = hotel_df.loc[index, "Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}" 

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params).json()
    
    # Convert the API response to JSON format
    # name_address = name_address.json()    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
sao francisco - nearest hotel: Hotel Atalaia
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
enewetak - nearest hotel: No hotel found
bac lieu - nearest hotel: Khách sạn Bạc Liêu
lata - nearest hotel: Residencial Botânico
nouadhibou - nearest hotel: Hotel valencia
tanout - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
eraan - nearest hotel: Trucut Travellers Inn
misratah - nearest hotel: الفندق السياحى
simanggang - nearest hotel: Mega Inn
bardai - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
senador jose porfirio - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
ebon - nearest hotel: No hotel found
nsunga - nearest hotel: Optima Lodges
hithadhoo - nearest hotel: Pebbles Inn
gizo - nearest hotel: Gizo Hotel
pointe aux piments - nearest hotel: Hotel Villas Mon Plaisir
at taj - nearest hotel: No hotel found
istgah-e rah ahan-e garmsar - nearest hotel: هتل شهر 

,City,Country,Lng,Lat,Humidity,Hotel Name
1,sao francisco,BR,-44.8644,-15.9486,35,Hotel Atalaia
4,lazaro cardenas,MX,-102.2000,17.9583,90,Hotel Sol del Pacífico
7,enewetak,MH,162.3373,11.3474,77,No hotel found
8,bac lieu,VN,105.7244,9.2850,82,Khách sạn Bạc Liêu
10,lata,PT,-8.3327,40.1629,37,Residencial Botânico
...,...,...,...,...,...,...
551,timbuktu,ML,-3.0074,16.7735,11,Hotel La Maison
555,taoudenni,ML,-3.9836,22.6783,9,No hotel found
556,mandera,KE,41.8670,3.9366,32,Kornesh Hotel and Lodging
557,embarcacion,AR,-64.0960,-23.2090,36,Punta Norte


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng", "Lat", geo=True, tiles = "OSM", size = "Humidity", color = "City", scale = 1, frame_width = 900, frame_height = 600, hover_cols='all', use_index=False 
)
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)